In [1]:
from glob import glob
from skimage.segmentation import slic
from PIL import Image
import numpy as np

In [2]:
slics = []
N_SEGMENTS = 4000
MAX_ITER = 10
COMPOSITION = 'pca'

In [3]:
img_paths = sorted(glob(f'scenes_{COMPOSITION}/*'))
truth_paths = sorted(glob('truth_masks/*'))
for idx, path in enumerate(img_paths):
    img = Image.open(path)
    img = np.array(img)
    truth = np.load(truth_paths[idx])
    
    mask = np.where(((truth != 0) & (truth != 255)), 1, 0) * 255

    print(f'Generating slic for image #{idx + 1}')
    slic_results = slic(img, n_segments=N_SEGMENTS, max_num_iter=MAX_ITER, mask=mask.squeeze(), convert2lab=True)

    slics.append(slic_results)

Generating slic for image #1
Generating slic for image #2
Generating slic for image #3
Generating slic for image #4
Generating slic for image #5
Generating slic for image #6
Generating slic for image #7
Generating slic for image #8
Generating slic for image #9
Generating slic for image #10


In [4]:
for idx, slic in enumerate(slics):
    np.save(f'slics/slic_x{idx + 1 :02d}-{COMPOSITION}.npy', slic)
    print(f'saved: slics/slic_x{idx + 1 :02d}-{COMPOSITION}.npy')

saved: slics/slic_x01-pca.npy
saved: slics/slic_x02-pca.npy
saved: slics/slic_x03-pca.npy
saved: slics/slic_x04-pca.npy
saved: slics/slic_x05-pca.npy
saved: slics/slic_x06-pca.npy
saved: slics/slic_x07-pca.npy
saved: slics/slic_x08-pca.npy
saved: slics/slic_x09-pca.npy
saved: slics/slic_x10-pca.npy
